In [ ]:
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm

In [ ]:
file_path = "extraction/data_censored.csv"

# Read the CSV file into a DataFrame
try:
    data_censored = pd.read_csv(file_path)
    print("Data loaded successfully!")
    print(data_censored.head())  # Display the first few rows
except FileNotFoundError:
    print(f"File not found at {file_path}")

In [ ]:
# Define directories for saving results
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)

trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

In [ ]:

# Define a function to structure the trial data
def set_data(trial_name, data, id_col, period_col, treatment_col, outcome_col, eligible_col):
    """Prepare a dictionary to structure trial data."""
    return {
        "trial_name": trial_name,
        "data": data,
        "id": data[id_col],
        "period": data[period_col],
        "treatment": data[treatment_col],
        "outcome": data[outcome_col],
        "eligible": data[eligible_col],
    }

# Per-Protocol (PP)
trial_pp = set_data(
    trial_name="PP",
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible",
)

# Intention-To-Treat (ITT)
trial_itt = set_data(
    trial_name="ITT",
    data=data_censored,
    id_col="id",
    period_col="period",
    treatment_col="treatment",
    outcome_col="outcome",
    eligible_col="eligible",
)

# Print the structured ITT trial data
print(trial_itt)

In [ ]:
# Define directory for saving models
trial_pp_dir = os.path.join(os.getcwd(), "trial_pp")
os.makedirs(trial_pp_dir, exist_ok=True)

# Separate data for treatment = 1 and treatment = 0 in the previous period
data_treated = data_censored[data_censored['treatment'].shift(1) == 1]
data_untreated = data_censored[data_censored['treatment'].shift(1) == 0]

# Define function to fit logistic regression models
def fit_logit_model(data, formula, save_path):
    """Fits a logistic regression model and saves it."""
    y = data['treatment']  # Dependent variable (treatment in current period)
    X = data[formula]  # Independent variables
    X = sm.add_constant(X)  # Add intercept term
    
    model = sm.Logit(y, X).fit()
    
    # Save model summary
    with open(save_path, "w") as f:
        f.write(model.summary().as_text())
    
    return model

# Fit numerator model (only age as predictor)
numerator_model = fit_logit_model(data_censored, ["age"], os.path.join(trial_pp_dir, "switch_numerator_model.txt"))

# Fit denominator model (age + x1 + x3 as predictors)
denominator_model = fit_logit_model(data_censored, ["age", "x1", "x3"], os.path.join(trial_pp_dir, "switch_denominator_model.txt"))

# Compute stabilized weights
data_censored["numerator_prob"] = numerator_model.predict(sm.add_constant(data_censored[["age"]]))
data_censored["denominator_prob"] = denominator_model.predict(sm.add_constant(data_censored[["age", "x1", "x3"]]))
data_censored["switch_weight"] = data_censored["numerator_prob"] / data_censored["denominator_prob"]

# Print first few switch weights
print(data_censored[["id", "switch_weight"]].head())

In [ ]:

# Define directories for saving models

trial_itt_dir = os.path.join(os.getcwd(), "trial_itt")
os.makedirs(trial_itt_dir, exist_ok=True)

# Define function to fit logistic regression models
def fit_logit_model(data, formula_vars, dependent_var, save_path):
    """Fits a logistic regression model and saves it."""
    y = data[dependent_var]  # Dependent variable (censoring event)
    X = data[formula_vars]  # Independent variables
    X = sm.add_constant(X)  # Add intercept term
    
    model = sm.Logit(y, X).fit()
    
    # Save model summary
    with open(save_path, "w") as f:
        f.write(model.summary().as_text())
    
    return model

# Fit numerator model: 1 - censored ~ x2
numerator_model_pp = fit_logit_model(
    data_censored, 
    ["x2"], 
    "censored", 
    os.path.join(trial_pp_dir, "censor_numerator_model.txt")
)

# Fit denominator model: 1 - censored ~ x2 + x1
denominator_model_pp = fit_logit_model(
    data_censored, 
    ["x2", "x1"], 
    "censored", 
    os.path.join(trial_pp_dir, "censor_denominator_model.txt")
)

# Compute stabilized censoring weights
data_censored["censor_numerator_prob"] = numerator_model_pp.predict(sm.add_constant(data_censored[["x2"]]))
data_censored["censor_denominator_prob"] = denominator_model_pp.predict(sm.add_constant(data_censored[["x2", "x1"]]))
data_censored["censor_weight"] = data_censored["censor_numerator_prob"] / data_censored["censor_denominator_prob"]

# Print first few censoring weights
print(data_censored[["id", "censor_weight"]].head())

# Fit numerator model: 1 - censored ~ x2 + assigned_treatment
numerator_model_itt = fit_logit_model(
    data_censored, 
    ["x2", "eligible"], 
    "censored", 
    os.path.join(trial_itt_dir, "censor_numerator_model.txt")
)

# Fit denominator model: 1 - censored ~ x2 + x1 + assigned_treatment
denominator_model_itt = fit_logit_model(
    data_censored, 
    ["x2", "x1", "eligible"],  
    "censored", 
    os.path.join(trial_itt_dir, "censor_denominator_model.txt")
)

# Compute weights for ITT
data_censored["censor_numerator_prob_itt"] = numerator_model_itt.predict(sm.add_constant(data_censored[["x2", "eligible"]]))
data_censored["censor_denominator_prob_itt"] = denominator_model_itt.predict(sm.add_constant(data_censored[["x2", "x1", "eligible"]]))
data_censored["censor_weight_itt"] = data_censored["censor_numerator_prob_itt"] / data_censored["censor_denominator_prob_itt"]

# Print ITT censoring weights
print(data_censored[["id", "censor_weight_itt"]].head())

In [ ]:
# Fit censoring model (numerator)
X_n = sm.add_constant(data_censored[['x2']])  # Predictor variables
y_n = 1 - data_censored['censored']           # Outcome variable (not censored)
censor_model_n = sm.Logit(y_n, X_n).fit()
print(censor_model_n.summary())

# Fit censoring model (denominator)
X_d = sm.add_constant(data_censored[['x2', 'x1']])  
y_d = 1 - data_censored['censored']  
censor_model_d = sm.Logit(y_d, X_d).fit()
print(censor_model_d.summary())

# Fit treatment switching model (numerator)
X_tn = sm.add_constant(data_censored[['age']])  
y_tn = data_censored['treatment']  
switch_model_n = sm.Logit(y_tn, X_tn).fit()
print(switch_model_n.summary())

# Fit treatment switching model (denominator)
X_td = sm.add_constant(data_censored[['age', 'x1', 'x3']])  
y_td = data_censored['treatment']  
switch_model_d = sm.Logit(y_td, X_td).fit()
print(switch_model_d.summary())

In [ ]:
# Fit treatment switching model (numerator)
X_tn = sm.add_constant(data_censored[['age']])  
y_tn = data_censored['treatment']  
switch_model_n = sm.Logit(y_tn, X_tn).fit()
print(switch_model_n.summary())

# Fit treatment switching model (denominator)
X_td = sm.add_constant(data_censored[['age', 'x1', 'x3']])  
y_td = data_censored['treatment']  
switch_model_d = sm.Logit(y_td, X_td).fit()
print(switch_model_d.summary())

In [ ]:
# Define predictors
X = sm.add_constant(data_censored[['x2']])  # Adjusting for x2
y = data_censored['outcome']  

# Fit logistic regression model
outcome_model = sm.Logit(y, X).fit()
print(outcome_model.summary())

In [ ]:
# Compute final stabilized weights
data_censored["final_weight_pp"] = data_censored["switch_weight"] * data_censored["censor_weight"]
data_censored["final_weight_itt"] = data_censored["switch_weight"] * data_censored["censor_weight_itt"]

# Print first few rows to compare
print(data_censored[["id", "final_weight_pp", "final_weight_itt"]].head())

In [ ]:
def expand_trials(data, max_followup=10, trial_type="PP"):
    expanded_rows = []
    
    # Select weight column based on trial type
    if trial_type == "PP":
        weight_col = "final_weight_pp"
    elif trial_type == "ITT":
        weight_col = "final_weight_itt"
    else:
        raise ValueError("Invalid trial type. Choose 'PP' or 'ITT'.")

    for _, row in data.iterrows():
        for t in range(max_followup):  
            expanded_rows.append({
                'id': row['id'],
                'trial_period': t,
                'followup_time': t,
                'outcome': row['outcome'],  
                'weight': row[weight_col],  # Use correct weight column
                'treatment': row['treatment'],
                'x2': row['x2'],
                'age': row['age'],
                'assigned_treatment': row['treatment']
            })
    
    return pd.DataFrame(expanded_rows)

# Expand PP trial
expanded_data_pp = expand_trials(data_censored, trial_type="PP")

# Expand ITT trial
expanded_data_itt = expand_trials(data_censored, trial_type="ITT")

# Check output
print(expanded_data_pp.head())
print(expanded_data_itt.head())

In [ ]:
print(data_censored.columns)

In [ ]:
def load_expanded_data(data, seed=1234, p_control=0.5):
    np.random.seed(seed)

    # If p_control < 1, randomly drop some outcome == 0 rows
    if p_control < 1:
        control_mask = (data["outcome"] == 0)  # Identify control cases
        sample_mask = control_mask & (np.random.rand(len(data)) > p_control)
        data = data[~sample_mask]  # Drop some controls
    
    return data

# Load ITT trial data with sampling
expanded_data_itt = load_expanded_data(expanded_data_itt, seed=1234, p_control=0.5)
expanded_data_pp = load_expanded_data(expanded_data_pp, seed=1234, p_control=0.5)

In [ ]:
def fit_msm(data, weight_cols):
    """ Fits a logistic regression model (MSM) using stabilized inverse probability weights. """
    
    # Winsorization function to cap extreme weights at 99th percentile
    def modify_weights(weights):
        q99 = np.quantile(weights, 0.99)
        return np.minimum(weights, q99)  # Cap weights at 99th percentile

    # Get weights and apply winsorization
    weights = modify_weights(data[weight_cols].sum(axis=1))  

    # Define independent variables (formula-like structure)
    data["followup_time_sq"] = data["followup_time"] ** 2
    data["trial_period_sq"] = data["trial_period"] ** 2

    independent_vars = ["assigned_treatment", "x2", "followup_time", "followup_time_sq", "trial_period", "trial_period_sq"]
    X = sm.add_constant(data[independent_vars])  # Add intercept
    y = data["outcome"]  # Dependent variable

    # Fit logistic regression (equivalent to glm with binomial logit in R)
    model = sm.GLM(y, X, family=sm.families.Binomial(), freq_weights=weights).fit()
    
    return model

# Fit MSM on ITT trial data
msm_model_pp = fit_msm(expanded_data_pp, weight_cols=["weight"])
msm_model_itt = fit_msm(expanded_data_itt, weight_cols=["weight"])

# Print model summary (like R's `trial_itt@outcome_model`)
print(msm_model_pp.summary())
print(msm_model_itt.summary())

In [ ]:
# Variance-covariance matrix
vcov_matrix_itt = msm_model_itt.cov_params()
# vcov_matrix_pp = msm_model_pp.cov_params()
# print(vcov_matrix_pp)
print(vcov_matrix_itt)

In [ ]:
from lifelines import KaplanMeierFitter

# Step 1: Fit a Kaplan-Meier Survival Model for ITT trial
kmf_treated = KaplanMeierFitter()
kmf_control = KaplanMeierFitter()

# Subset data for assigned_treatment = 1 (treated) and 0 (control)
data_treated = expanded_data_itt[expanded_data_itt["assigned_treatment"] == 0]
data_control = expanded_data_itt[expanded_data_itt["assigned_treatment"] == 1]

# Fit Kaplan-Meier survival curves
kmf_treated.fit(data_treated["followup_time"], event_observed=data_treated["outcome"], label="Treated")
kmf_control.fit(data_control["followup_time"], event_observed=data_control["outcome"], label="Control")

# Step 2: Predict survival probabilities over time
followup_times = np.arange(0, 11)  # Predict for 0 to 10 follow-up periods
survival_treated = kmf_treated.survival_function_at_times(followup_times)
survival_control = kmf_control.survival_function_at_times(followup_times)

# Compute the survival difference
survival_diff = survival_treated - survival_control
# Step 3: Plot Survival Difference
plt.figure(figsize=(8, 5))
plt.plot(followup_times, survival_diff, label="Survival Difference", color="blue")

# Confidence intervals (assuming normal approximation)
std_error = np.sqrt(survival_treated * (1 - survival_treated) / len(data_treated) +
                    survival_control * (1 - survival_control) / len(data_control))

ci_lower = survival_diff - 1.96 * std_error
ci_upper = survival_diff + 1.96 * std_error

# Add confidence interval lines
plt.plot(followup_times, ci_lower, color="red", linestyle="dashed", label="2.5% CI")
plt.plot(followup_times, ci_upper, color="red", linestyle="dashed", label="97.5% CI")

# Labels and legend
plt.xlabel("Follow-up Time")
plt.ylabel("Survival Difference")
plt.title("Survival Difference Over Time")
plt.legend()
plt.show()